In [1]:
import pandas as pd
import numpy as np
import random
from autocorrect import Speller
import re
from LLM import LLM
from PromptBuilder import PromptBuilder
from metadata import language_registers, discussion_tones

2025-04-04 16:29:35.543602: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-04 16:29:35.554075: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743776975.567239 3900076 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743776975.571246 3900076 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-04 16:29:35.585037: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

### Biollama :

In [2]:
model = LLM(model_name="iRASC/BioLlama-Ko-8B")

/home/laajila/miniconda3/envs/darts_env/lib/python3.11/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

ValueError: Trying to set a tensor of shape torch.Size([1024, 4096]) in "weight" (which has shape torch.Size([4096, 4096])), this looks incorrect.

### Example : 

In [19]:
df['Prompt'][79]

[{'role': 'system',
  'content': 'You are a chatbot that writes in the voice of a patient using a Formal Register register with a Fearful tone.'},
 {'role': 'user',
  'content': "Write a phrase from a sick patient describing their symptoms. The symptoms the patient describes are: Glaucoma, Intraoperative renal injury. The patient speaks in Formal Register Register and uses a Fearful tone. The grades of the symptoms correspond to a scale ['Grade 4', 'Grade 1'] (in the same order as the symptoms list, where 1 corresponds to the least severe and 5 to the most severe). These grades correspond to: ['Visual field deficit within the central 10 degrees of the visual field in the affected eye', 'Primary repair of injured organ/structure indicated']. The description should be very detailed and comprehensive.  Include spelling mistakes in the generated phrase. Respond with only the sentence in the patient's voice, enclosed in double quotes, without any additional text, comments, or notes."}]

In [18]:
df['Dialogue_Generated'][79]

' "I have been experiencing some vision problems lately, and I was told that I have glaucoma. I am very concerned about this because I have heard that it can lead to blindness. I also had a kidney injury during a recent surgery, and I was told that I have intraoperative renal injury. I am very worried about this because I have heard that it can lead to chronic kidney disease. I am hoping that you can help me with this problem."'

### Data generated :

In [5]:
with_biollama = pd.read_csv('building_dataset_with_biollama8B.csv')
with_biollama.head(5)

,Unnamed: 0,Symptoms,Dialogue_Generated,Language_style,Tone,Detail_level,Enumeration,Explicit_symptom,Spelling_errors,Prompt
0,0,['Anemia'],"""My blood is as dry as the desert sand, and m...",Poetic/Literary Register,Fearful,4,False,False,False,"[{'role': 'system', 'content': 'You are a chat..."
1,1,['Blood and lymphatic system disorders - Other...,"""I am experiencing a severe Blood and lymphat...",Formal Register,Insulting,2,False,True,True,"[{'role': 'system', 'content': 'You are a chat..."
2,2,"['Bone marrow hypocellular', 'Hypernatremia']","""I've been feeling really weak lately and I'v...",Informal Register,Friendly,4,False,False,False,"[{'role': 'system', 'content': 'You are a chat..."
3,3,['Disseminated intravascular coagulation'],"""I've got this weird feeling in my body, like...",Informal Register,Confused,2,False,False,False,"[{'role': 'system', 'content': 'You are a chat..."
4,4,['Eosinophilia'],"""I have a severe Eosinophilia.""",Formal Register,Angry,1,False,True,False,"[{'role': 'system', 'content': 'You are a chat..."


In [6]:
with_llama = pd.read_csv('building_dataset_with_llama8B.csv')
with_llama.head(5)

,Unnamed: 0,Symptoms,Dialogue_Generated,Language_style,Tone,Detail_level,Enumeration,Explicit_symptom,Spelling_errors,Prompt
0,0,['Anemia'],"""Alas, my weary soul, beset by the weight of a...",Poetic/Literary Register,Friendly,2,False,True,False,"[{'role': 'system', 'content': 'You are a chat..."
1,1,['Blood and lymphatic system disorders - Other...,"""Alas, my pitiful physician, I'm beset by the ...",Poetic/Literary Register,Insulting,4,False,True,False,"[{'role': 'system', 'content': 'You are a chat..."
2,2,['Bone marrow hypocellular'],"""I'm not really sure what's going on, but my d...",Neutral/Standard Register,Confused,2,False,False,False,"[{'role': 'system', 'content': 'You are a chat..."
3,3,['Disseminated intravascular coagulation'],"""Great, just what I needed, a lovely case of D...",Neutral/Standard Register,Insulting,4,False,True,False,"[{'role': 'system', 'content': 'You are a chat..."
4,4,['Eosinophilia'],"""Oh, the terror that grips my fragile form, as...",Poetic/Literary Register,Fearful,3,False,False,False,"[{'role': 'system', 'content': 'You are a chat..."


In [10]:
from typing import List

class PromptBuilder:
    def _init_(self) -> None:
        """
        Initialize the prompt generator.
        """
        pass


    def build_prompt(
            self,
            symptoms : list , 
            description : list ,
            meta : list , 
            detail_level: float, 
            enumeration: bool = False, 
            explicit_symptom: bool = False, 
            language_style: str = "everyday language", 
            spelling_errors: bool = False, 
            tone: str = "neutral"

            ) -> list[dict]:
        
        """
        Construct a prompt based on the given parameters.
        """

        if not isinstance(symptoms, list) or not all(isinstance(s, str) for s in symptoms):
            raise ValueError("symptoms must be a list of strings.")
        

        messages = [
            {
                "role": "system",
                "content": f"You are a chatbot that writes in the voice of a patient using a {language_style} register with a {tone} tone."
            },
            {
                "role": "user",
                "content": (
                    f"Write a phrase from a sick patient describing their symptoms. The symptoms the patient describes are: {', '.join(symptoms)}. "
                    f"The patient speaks in {language_style} Register and uses a {tone} tone. "
                )
            }
        ]
        
        for i in range(len(description)) :

            messages[1]['content'] += (

                f"The {description[i]} of the symptom correspond to {meta[i]} "
            )
        
        
        # Specify the level of detail
        detail_instructions = {
            1: "The description should be very brief with minimal details.",
            2: "The description should be brief with some basic details.",
            3: "The description should provide a moderate level of detail.",
            4: "The description should be detailed and thorough.",
            5: "The description should be very detailed and comprehensive."
        }
        messages[1]['content'] += detail_instructions.get(detail_level, "Provide a moderate level of detail.") + " "

        if enumeration:
            messages[1]['content'] += (
                "The patient should just enumerate the descriptions. For example, for symptom Anemia, a sentence could be: "
                "'Fatigue and Weakness, Pale Skin, Shortness of Breath, Dizziness, Irregular Heartbeat, Chest Pain, Cold Hands and Feet, Headaches'. "
            )

        if explicit_symptom:
            messages[1]['content'] += f"The patient should explicitly mention the symptoms they have (e.g., {', '.join(symptoms)}). "

        if spelling_errors:
            messages[1]['content'] += " Include spelling mistakes in the generated phrase."

        messages[1]['content'] += " Respond with only the sentence in the patient's voice, enclosed in double quotes, without any additional text, comments, or notes."

        return messages

### PRO dataset :

In [3]:
a = pd.read_excel('PRO-CTCAE_Questionnaire_Terminology.xls', sheet_name = 'PRO')
a.head(4)

,NCIt Code of Subset,Subset Name,NCIt Concept Code,NCIt PT,NCIt Definition,PRO-CTCAE PT,Has PRO-CTCAE Attribute Code,Has PRO-CTCAE Attribute PT,PRO-CTCAE Attribute Has Value Code,PRO-CTCAE Value PT
0,C205862,PRO-CTCAE Oral Adverse Event Terminology,C205929,Cracking at the Corners of the Mouth (Cheilosi...,Cheilosis or cheilitis as recorded on the PRO-...,Cracking at the corners of the mouth (cheilosi...,C25676,Severity,NaN,NaN
1,C205862,PRO-CTCAE Oral Adverse Event Terminology,C205927,"Difficulty Swallowing, PRO-CTCAE",Problems with swallowing as recorded on the PR...,Difficulty Swallowing,C25676,Severity,NaN,NaN
2,C205862,PRO-CTCAE Oral Adverse Event Terminology,C205926,"Dry Mouth, PRO-CTCAE",Dryness of the oral mucosa secondary to a decr...,Dry Mouth,C25676,Severity,NaN,NaN
3,C205862,PRO-CTCAE Oral Adverse Event Terminology,C205931,"Hoarseness, PRO-CTCAE",A raspy alteration in one's voice as recorded ...,Hoarseness,C25676,Severity,NaN,NaN


In [4]:
index = (a [a['PRO-CTCAE PT'] == 'Pain and swelling at injection site']).index.values[0] +1 
symptoms = a.iloc[:index]['PRO-CTCAE PT'].values
symptoms

array(['Cracking at the corners of the mouth (cheilosis/cheilitis)',
       'Difficulty Swallowing', 'Dry Mouth', 'Hoarseness',
       'Mouth/throat sores', 'Voice quality changes', 'Abdominal pain',
       'Bloating', 'Constipation', 'Decreased appetite', 'Diarrhea',
       'Fecal incontinence', 'Gas', 'Heartburn', 'Hiccups', 'Nausea',
       'Taste Changes', 'Vomiting', 'Coughing', 'Shortness of breath',
       'Wheezing', 'Heart palpitations', 'Swelling (arms or legs)',
       'Acne', 'Bed/pressure sores', 'Hair loss',
       'Hand-foot syndrome (a rash of the hands and feet that can cause cracking, peeling, redness or pain)',
       'Hives', 'Itching', 'Nail discoloration', 'Nail loss',
       'Nail ridging', 'Radiation skin reaction', 'Rash',
       'Sensitivity to sunlight', 'Skin darkening', 'Skin dryness',
       'Stretch marks', 'Dizziness', 'Numbness & tingling',
       'Blurred vision', 'Flashing lights', 'Ringing in ears',
       'Visual floaters', 'Watery eyes', 'Concentra

In [5]:
Dict = {}
for symptom in symptoms : 
    try:
        Dict[symptom] = {}
        descriptions_code = a[a['PRO-CTCAE PT'] == symptom]['Has PRO-CTCAE Attribute Code'].values[0].split(" || ")
        descriptions = a [a['PRO-CTCAE PT'] == symptom]['Has PRO-CTCAE Attribute PT'].values[0].split(" || ")
        for description in descriptions :
            Dict[symptom][description]  = a[a['PRO-CTCAE PT'] == description  ]['PRO-CTCAE Value PT'].values[0].split(" || ")
    
    except Exception as e :
        print(e)

'float' object has no attribute 'split'


In [6]:
Dict

{'Cracking at the corners of the mouth (cheilosis/cheilitis)': {'Severity': ['Very severe',
   'Not sexually active',
   'Prefer not to answer',
   'None',
   'Not applicable',
   'Moderate',
   'Mild',
   'Severe']},
 'Difficulty Swallowing': {'Severity': ['Very severe',
   'Not sexually active',
   'Prefer not to answer',
   'None',
   'Not applicable',
   'Moderate',
   'Mild',
   'Severe']},
 'Dry Mouth': {'Severity': ['Very severe',
   'Not sexually active',
   'Prefer not to answer',
   'None',
   'Not applicable',
   'Moderate',
   'Mild',
   'Severe']},
 'Hoarseness': {'Severity': ['Very severe',
   'Not sexually active',
   'Prefer not to answer',
   'None',
   'Not applicable',
   'Moderate',
   'Mild',
   'Severe']},
 'Mouth/throat sores': {'Interference (with daily activities)': ['Not at all',
   'A little bit',
   'Somewhat',
   'Quite a bit',
   'Very much'],
  'Severity': ['Very severe',
   'Not sexually active',
   'Prefer not to answer',
   'None',
   'Not applicable',

In [9]:
Dict['Nosebleed'].values()

dict_values([['Almost constantly', 'Not sexually active', 'Prefer not to answer', 'Occasionally', 'Frequently', 'Never', 'Rarely'], ['Very severe', 'Not sexually active', 'Prefer not to answer', 'None', 'Not applicable', 'Moderate', 'Mild', 'Severe']])

In [12]:
data = []

for symptom in Dict.keys():

        detail_level = np.random.choice([1,2,3,4,5])

        enumeration = np.random.choice([True, False], p=[0.2, 0.8])

        explicit_symptom = np.random.choice([True, False], p=[0.2, 0.8])

        language_style = random.choice(language_registers)['name']

        tone = random.choice(discussion_tones)['name']

        spelling_errors = random.choice([True, False])

        prompt = PromptBuilder().build_prompt([symptom], list(Dict[symptom].keys()), list(Dict[symptom].values()), detail_level, enumeration, explicit_symptom, language_style, spelling_errors, tone) 
    
        phrase_generated = model.generate_text(messages = prompt)

        data.append([phrase_generated , symptom, Dict[symptom].keys(), Dict[symptom].values(), language_style, tone, detail_level, enumeration, explicit_symptom, spelling_errors])

        df = pd.DataFrame(data, columns=["Dialogue_Generated", "symptom", "description", "meta", "language_style", "Tone", "Detail_level", "Enumeration", "Explicit_symptom", "Spelling_errors"])

NameError: name 'model' is not defined

In [9]:
df

,Dialogue_Generated,symptom,description,meta,language_style,Tone,Detail_level,Enumeration,Explicit_symptom,Spelling_errors
0,"""I have this cracking at the corners of my mo...",Cracking at the corners of the mouth (cheilosi...,Severity,Very severe,Neutral/Standard Register,Confused,2,False,False,True
1,"""I have been noticing that the corners of my ...",Cracking at the corners of the mouth (cheilosi...,Severity,Not sexually active,Neutral/Standard Register,Confused,5,False,False,True
2,"""I have cracks at the corners of my mouth.""",Cracking at the corners of the mouth (cheilosi...,Severity,Prefer not to answer,Informal Register,Neutral,1,False,False,False
3,"""I am fed up with this cracking at the corner...",Cracking at the corners of the mouth (cheilosi...,Severity,None,Formal Register,Angry,2,False,False,False
4,"""I'm so scared, I have this weird cracking at...",Cracking at the corners of the mouth (cheilosi...,Severity,Not applicable,Informal Register,Fearful,1,False,False,False
...,...,...,...,...,...,...,...,...,...,...
818,"""My arm is all puffy and aching at the spot w...",Pain and swelling at injection site,Presence/Absence,Not sexually active,Poetic/Literary Register,Confused,2,False,False,True
819,"""I got this shot in my arm and now it's all s...",Pain and swelling at injection site,Presence/Absence,Prefer not to answer,Vulgar Register,Confused,5,False,False,True
820,"""My arm is aching and inflamed at the injecti...",Pain and swelling at injection site,Presence/Absence,Not applicable,Poetic/Literary Register,Neutral,2,False,False,True
821,"""I have severe pain and swelling at the injec...",Pain and swelling at injection site,Presence/Absence,No,Formal Register,Fearful,1,True,False,False


In [11]:
df['Dialogue_Generated'][1]

' "I have been noticing that the corners of my mouth crack and bleed sometimes. I thought it was just dry mouth from my medication but it seems to be getting worse. I have been trying to keep my mouth moist by drinking water but it doesn\'t seem to be helping. I have also been trying to avoid spicy foods but I still eat them sometimes. I don\'t know what is causing this but I hope I can find out."'

In [12]:
df.to_csv('PRO_building_dataset_with_biollama8B.csv')

In [16]:
# symptoms_data = [
    
#     {'symptom': 'head ache', 'frequency': ['always', 'sometimes', 'rarely']},
#     {'symptom': 'fatigue', 'frequency': ['always', 'sometimes']},
#     {'symptom': 'nausea', 'frequency': ['rarely', 'sometimes']},
#     {'symptom': 'dizziness', 'frequency': ['often', 'sometimes']},
#     {'symptom': 'cough', 'frequency': ['always', 'sometimes']},
#     {'symptom': 'fever', 'frequency': ['rarely']},
#     {'symptom': 'shortness of breath', 'frequency': ['always']},
#     {'symptom': 'chest pain', 'frequency': ['never', 'rarely']},
#     {'symptom': 'back pain', 'frequency': ['sometimes', 'often']},
#     {'symptom': 'sore throat', 'frequency': ['sometimes', 'rarely']}
# ]

In [17]:
# p = [0.4, 0.3, 0.2, 0.1]
# c = [1, 2, 3, 4]

# symptoms_freq = []

# for el in symptoms_data:

#     el_copy = list(CTCAE_symptoms.copy())

#     CTCAE_symptoms_copy.remove(symptom) 

#     proba = np.random.choice(c, p)
#     symptom2 = np.random.choice(symptoms_data)